In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
# Константы
t21 = 10e-3
h = 6.6e-34
Aeff = 3.14 * (2e-6)**2
Gp = 0.83
Gs = 0.5311143935168284
C = 3e8
hs = 1550e-9
hp = 980e-9
Ner = 1.9e25
alphap = 0.006
alphas = 0.0
l = 3
dhs = 0.16e-9
sigma12S = 3.198e-25
sigma21S = 4.4e-25
sigma12P = 2.245e-25
sigma21P = 0.0
z0 = 0
Z = 100
H = l / Z
zzz = np.linspace(z0, l, Z)
#R1 = 0.9
R2 = 0.9 # Выход
x1 = 0.001
x2 = 0.01
epsilon = 1e-6  # Точность для вычислений
count = 0
Pp = float(input("Введите начальную мощность накачки Pp0: "))
# Функции для расчета изменений концентраций и мощностей
def dN2(z, N2, N1, Pp, Ps):
    return Gp * hp / (h * C * Aeff ) * (sigma12P * N1 - sigma21P * N2) * Pp + Gs * hp / (h * C * Aeff) * \
           (sigma12S * N1 - sigma21S * N2) * Ps - N2 / t21
def dN1(z, N2, N1, Pp, Ps):
    return -(Gp * hp / (h * C * Aeff ) * (sigma12P * N1 - sigma21P * N2) * Pp + Gs * hp / (h * C * Aeff) * \
            (sigma12S * N1 - sigma21S * N2) * Ps - N2 / t21)
def dPp(z, N2, N1, Pp):
    return Gp * (sigma21P * N2 - sigma12P * N1) * Pp - alphap * Pp
def dPs(z, N2, N1, Ps):
    return Gs * (sigma21S * N2 - sigma12S * N1) * Ps - alphas * Ps 
# Объединяем функции в одну для использования в solve_ivp
def system(z, y):
    N2, N1, Pp, Ps = y
    return np.array([dN2(z, N2, N1, Pp, Ps), dN1(z, N2, N1, Pp, Ps), dPp(z, N2, N1, Pp), dPs(z, N2, N1, Ps)])
# Функция для вычисления ошибки
while abs(x2 - x1) > epsilon and (count < 100):
    # Решение с первой догадкой мощности
    sol1 = solve_ivp(system, [0, l], [0.0, Ner, Pp, x1], method='RK45', max_step=H)
    Ps1 = sol1.y[3, -1]
    
    # Решение с второй догадкой мощности
    sol2 = solve_ivp(system, [0, l], [0.0, Ner ,Pp, x2], method='RK45', max_step=H)
    Ps2 = sol2.y[3, -1]
    
    # Расчет отраженной мощности
    P01 = Ps1 * R2
    P02 = Ps2 * R2
    
    # Обновление значений мощности сигнала с использованием метода секанта
    m1 = P01 - x1
    m2 = P02 - x2
    tmp = x2
    x2 = x1 - ((x1 - x2) / (m1 - m2) * m1)
    x1 = tmp
    count += 1
# Получаем итоговые значения для мощности сигнала и накачки
print(f"Начальное значение Ps_guess: {x2}")

Введите начальную мощность накачки Pp0:  80e-3


Начальное значение Ps_guess: 0.6606941693230926
